# Hospital Staffing Optimization with Genetic Algorithms

This notebook provides a comprehensive walkthrough of the Hospital Simulation system and the Genetic Algorithm (GA) used to optimize staffing schedules. Our goal is to minimize total operational costs, which include:
- **Staffing Costs**: Hourly wages and setup costs for temporary staff.
- **Waiting Costs**: Penalties for patients waiting too long for care.
- **Diversion Costs**: High penalties when the ER is at capacity and ambulances must be diverted.

## 1. Setup and Configuration

First, we import the necessary modules and load the simulation configuration. The configuration defines arrival rates, department capacities, costs, and patient flow probabilities.

In [ ]:
import sys
import os

# Ensure the project root is in the path
project_root = '/Users/shauryaguleria/Documents/School/Hospital Optimizer'
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import from the simulation package
try:
    from simulation.config import ARRIVAL_RATES, CAPACITY, COSTS, INITIAL_STAFF
except ImportError:
    # Fallback/Debug if something is wrong with the path
    print("Error importing config. Checking path...")
    print(sys.path)
    raise

import pandas as pd
import matplotlib.pyplot as plt

print("Configuration Loaded.")
print(f"Departments: {list(CAPACITY.keys())}")
print(f"Initial Staffing: {INITIAL_STAFF}")

### Key Configuration Highlights

The simulation is driven by several key parameters defined in `simulation/config.py`:

- **Arrival Rates**: Fluctuate by hour of day (e.g., higher arrivals in the morning/evening).
- **Costs**: 
    - **Wait**: Cost per hour a patient waits.
    - **Diversion**: $5,000 penalty for diverting an ambulance.
    - **Extra Staff**: $40/hr for temporary staff.
- **Patient Flow**: Probabilities determine how patients move from ER -> Surgery -> Critical Care -> etc.

---

## 2. The Hospital Simulation Model

The core simulation is implemented in `simulation/hospital.py` (formerly simulation.py). It uses `simpy`, a discrete-event simulation framework.

### How it works:
1. **Arrival Generator**: Creates patients based on Poisson distribution rates for each hour.
2. **Patient Flow**: 
    - Patients arrive at the **ER**.
    - If resources (Beds/Staff) are available, they are treated.
    - If not, they wait (accumulating Wait Cost) or are diverted (Diversion Cost).
    - After treatment, they may be discharged or transferred to **Surgery**, **CriticalCare**, or **StepDown**.
3. **Resource Management**: Each department has a limited number of **beds** and **staff** (Doctors/Nurses). 
    - Staff levels can change hourly based on the **Staffing Schedule**.

The `HospitalSimulation` class takes a `staffing_schedule` as input. If none is provided, it uses the static `INITIAL_STAFF` for all 24 hours.

In [ ]:
# Import simulation class from the package
from simulation.hospital import HospitalSimulation

# Run a Baseline Simulation (No Optimization)
print("Running Baseline Simulation with Initial Staffing...")
baseline_sim = HospitalSimulation(duration_hours=24, staffing_schedule=None)
baseline_cost = baseline_sim.run()

print(f"Baseline Total Cost: ${baseline_cost:,.2f}")

---

## 3. Genetic Algorithm (GA) Implementation

To find the best staffing schedule, we use a Genetic Algorithm. This is an optimization technique inspired by natural selection.

### Why GA?
The search space is huge. We have **24 hours**, and for each hour, we can adjust staff levels for **4 departments**. 
If we allow just ±2 staff variance per department per hour, that's $5^{4 \times 24}$ combinations! A brute force search is impossible. GA allows us to evolve towards a good solution efficiently.

### Components of our GA (`simulation/optimizer.py`)

1.  **Genome (The Solution)**:
    - A dictionary representing the schedule: `{Hour: {Dept: Count}}`.
    - Example: `{0: {'ER': 20, ...}, 1: {'ER': 18, ...}}`.

2.  **Population**:
    - A collection of random schedules (genomes). We start with a population size of 5.

3.  **Fitness Function (Evaluation)**:
    - The fitness of a schedule is the **Total Cost** returned by the simulation.
    - Lower Cost = Higher Fitness.
    - *Note*: Since simulation is stochastic (random arrivals), we run it multiple times (e.g., 5 iterations) and average the cost to get a stable score.

4.  **Selection**:
    - We sort the population by cost and keep the top 50% ("Survival of the Fittest").

5.  **Crossover (Reproduction)**:
    - We create new schedules by combining parts of two parent schedules.
    - Example: Take morning shifts from Parent A and evening shifts from Parent B.

6.  **Mutation**:
    - With a small probability (`MUTATION_RATE`), we slightly tweak a schedule (e.g., add/remove 1 staff member) to introduce variety and prevent getting stuck in local optima.

In [ ]:
# Import optimizer from the package
import importlib
import simulation.optimizer
importlib.reload(simulation.optimizer)
from simulation.optimizer import StaffingOptimizer

# Genetic Algorithm Parameters
POPULATION_SIZE = 5
GENERATIONS = 100
MUTATION_RATE = 0.1
ELITISM = 2

# Initialize the Optimizer
optimizer = StaffingOptimizer(
    population_size=POPULATION_SIZE,
    generations=GENERATIONS,
    mutation_rate=MUTATION_RATE,
    elitism=ELITISM
)

# Explain parameters
print(f"Population Size: {POPULATION_SIZE}")
print(f"Generations: {GENERATIONS}")
print(f"Mutation Rate: {MUTATION_RATE}")
print(f"Elitism: {ELITISM}")


---

## 4. Running the Optimization

We will now run the Genetic Algorithm. It will iterate through generations, evolving the population to minimize cost.

**Watch the output below:** You should see the "Best Cost" decrease (or stay stable if already optimal) compared to the Baseline.

In [ ]:
# Run the optimization
# This might take a minute as it runs multiple simulations per individual per generation.
best_schedule, min_cost = optimizer.run()

## 5. Results & Analysis

Let's analyze the best schedule found. We will look at how staffing levels vary throughout the day compared to the static baseline.

In [ ]:
print(f"\nOptimization Complete!")
print(f"Best Minimum Cost Found: ${min_cost:,.2f}")
print(f"Baseline Cost: ${baseline_cost:,.2f}")
print(f"Savings: ${baseline_cost - min_cost:,.2f}")

# Visualize the Best Schedule
import matplotlib.pyplot as plt
import numpy as np

hours = list(range(24))
er_staff = [best_schedule[h]['ER'] for h in hours]
surgery_staff = [best_schedule[h]['Surgery'] for h in hours]
critical_staff = [best_schedule[h]['CriticalCare'] for h in hours]
stepdown_staff = [best_schedule[h]['StepDown'] for h in hours]

hour_labels = [f"{h%12 or 12} {'AM' if h < 12 else 'PM'}" for h in hours]

# ER and Surgery Graph
plt.figure(figsize=(12, 6))
plt.plot(hours, er_staff, label='ER Staff (Optimized)', marker='o')
if 'INITIAL_STAFF' in globals():
    plt.axhline(y=INITIAL_STAFF['ER'], color='r', linestyle='--', label='ER Baseline')

# Trend line for ER
z_er = np.polyfit(hours, er_staff, 3)
p_er = np.poly1d(z_er)
plt.plot(hours, p_er(hours), "r:", alpha=0.7, label='ER Trend')

plt.plot(hours, surgery_staff, label='Surgery Staff (Optimized)', marker='x')
if 'INITIAL_STAFF' in globals():
    plt.axhline(y=INITIAL_STAFF['Surgery'], color='orange', linestyle='--', label='Surgery Baseline')

# Trend line for Surgery
z_surg = np.polyfit(hours, surgery_staff, 3)
p_surg = np.poly1d(z_surg)
plt.plot(hours, p_surg(hours), "k:", alpha=0.5, label='Surgery Trend')

plt.title('Optimized Staffing Schedule vs Baseline (ER & Surgery)')
plt.xlabel('Hour of Day')
plt.ylabel('Number of Staff')
plt.xticks(hours, hour_labels, rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Critical Care and Step Down Graph
plt.figure(figsize=(12, 6))
plt.plot(hours, critical_staff, label='Critical Care Staff (Optimized)', marker='o', color='purple')
if 'INITIAL_STAFF' in globals():
    plt.axhline(y=INITIAL_STAFF['CriticalCare'], color='purple', linestyle='--', label='Critical Care Baseline')

# Trend line for Critical Care
z_crit = np.polyfit(hours, critical_staff, 3)
p_crit = np.poly1d(z_crit)
plt.plot(hours, p_crit(hours), color="purple", linestyle=':', alpha=0.7, label='Critical Care Trend')

plt.plot(hours, stepdown_staff, label='Step Down Staff (Optimized)', marker='x', color='green')
if 'INITIAL_STAFF' in globals():
    plt.axhline(y=INITIAL_STAFF['StepDown'], color='green', linestyle='--', label='Step Down Baseline')

# Trend line for Step Down
z_step = np.polyfit(hours, stepdown_staff, 3)
p_step = np.poly1d(z_step)
plt.plot(hours, p_step(hours), color="green", linestyle=':', alpha=0.7, label='Step Down Trend')

plt.title('Optimized Staffing Schedule vs Baseline (Critical Care & Step Down)')
plt.xlabel('Hour of Day')
plt.ylabel('Number of Staff')
plt.xticks(hours, hour_labels, rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Display Full Best Schedule
print("Best Hourly Staffing Schedule:")
schedule_df = pd.DataFrame.from_dict(best_schedule, orient='index')

total_initial_staff = sum(INITIAL_STAFF.values())
schedule_df['Extra Staff'] = (schedule_df.sum(axis=1) - total_initial_staff).clip(lower=0).astype(int)

def format_hour(h):
    am_pm = "AM" if h < 12 else "PM"
    hour_12 = h if h <= 12 else h - 12
    if hour_12 == 0: hour_12 = 12
    return f"{h} ({hour_12} {am_pm})"

schedule_df.index = [format_hour(h) for h in schedule_df.index]
schedule_df.index.name = 'Hour'
display(schedule_df)